# Problem Set 4

In [1]:
#packages and API keys
import requests, json
import pandas as pd
import numpy
import math
# For linear regression
import statsmodels.formula.api as smf

## 1. Exploring the data

### 1.1 Upload the 1996 car data from my Github repository for the course (using the JSON format is recommended). Report the number of ob- servations (car makes) an the number of columns (variables) in the data you uploaded.

In [2]:
# car data from Github
url = "https://raw.githubusercontent.com/ArieBeresteanu/Econ1923_Fall2022/main/Demand%20Estimation/cardata1996.json"
data = json.loads(requests.get(url).text)
cars = pd.DataFrame(data)

In [3]:
# 1996 car data
cars

,car,year,firm_id,firm_name,division,model,segm1,Quantity,Price,wheel_base,length,width,weight,disp,hp,mpg_city,mpg_highway
0,0,1996,3,HONDA,Acura,SLX,32,2565,34320,108.7,178.9,72.4,4315,3.2,190,14,18
1,0,1996,19,GM,Chevrolet,Astro,21,125962,19152,110.0,189.8,77.5,3398,4.3,190,17,22
2,0,1996,19,GM,Chevrolet,Blazer,32,246307,17406,107.0,176.8,65.4,3365,4.3,165,16,21
3,0,1996,19,GM,Chevrolet,Express/VAN,22,80203,21987,135.0,218.7,79.2,4641,4.3,195,15,19
4,0,1996,19,GM,Chevrolet,S10,41,190178,10125,108.3,178.2,64.7,2773,2.2,118,23,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,1,1996,7,VOLKS,volkswagen,golf III,1,24208,13150,97.4,160.4,66.7,2529,2.0,115,23,28
181,1,1996,7,VOLKS,volkswagen,jetta III,2,85022,14250,97.4,173.4,66.7,2657,2.0,115,23,28
182,1,1996,7,VOLKS,volkswagen,passat,3,19850,18490,103.3,181.5,67.5,3097,2.8,172,21,27
183,1,1996,6,VOLVO,VOLVO,850,5,67636,26300,104.9,183.5,69.3,3232,2.4,168,20,26


In [4]:
# number of observations and columns
n_cars, n_cols = cars.shape

print(f"The data contains information on {n_cars} cars and it has {n_cols} columns.")

The data contains information on 185 cars and it has 17 columns.


In [5]:
# number of car makes
makes = cars['firm_name']
n_makes = makes.nunique()

print(f"The data contains information for {n_makes} car makes")

The data contains information for 21 car makes


### 1.2 Create a frequency table for the car manufacturers. How many car models are sold by each car manufacturer? (I’m referring to the firm name variable)

In [6]:
# frequency table of how many models each manufacturer sold
pd.crosstab(index=cars['firm_name'], columns='models')

col_0,models
firm_name,
BMW,4
CHRYSLER,21
FORD,25
GM,51
HONDA,9
HYUNDAI,3
ISUZU,4
JAGUAR,2
KIA,2


## 2. Initial analysis

### 2.1 Assume that the number of potential car buyers in a year equals the number of households in the US at that year. Create a new variable (i.e. a new column in the Dataframe) for the (natural) log of the market share of each car make in the data. In other words, divide the quantity sold by the number of households and take a log of this ratio.

In [7]:
# number of car buyers equals number of U.S. households in 1996
# 98857000 households in the US in 1996 according to Census data

# create new variable for ln(Quantity / number of households)

In [8]:
nHH96 = 98857000
cars['share'] = cars['Quantity'].map(lambda x: x/nHH96)

In [9]:
cars['logShare'] = cars['share'].map(lambda x: math.log(x))

In [10]:
cars

,car,year,firm_id,firm_name,division,model,segm1,Quantity,Price,wheel_base,length,width,weight,disp,hp,mpg_city,mpg_highway,share,logShare
0,0,1996,3,HONDA,Acura,SLX,32,2565,34320,108.7,178.9,72.4,4315,3.2,190,14,18,0.000026,-10.559471
1,0,1996,19,GM,Chevrolet,Astro,21,125962,19152,110.0,189.8,77.5,3398,4.3,190,17,22,0.001274,-6.665449
2,0,1996,19,GM,Chevrolet,Blazer,32,246307,17406,107.0,176.8,65.4,3365,4.3,165,16,21,0.002492,-5.994851
3,0,1996,19,GM,Chevrolet,Express/VAN,22,80203,21987,135.0,218.7,79.2,4641,4.3,195,15,19,0.000811,-7.116869
4,0,1996,19,GM,Chevrolet,S10,41,190178,10125,108.3,178.2,64.7,2773,2.2,118,23,28,0.001924,-6.253469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,1,1996,7,VOLKS,volkswagen,golf III,1,24208,13150,97.4,160.4,66.7,2529,2.0,115,23,28,0.000245,-8.314746
181,1,1996,7,VOLKS,volkswagen,jetta III,2,85022,14250,97.4,173.4,66.7,2657,2.0,115,23,28,0.000860,-7.058520
182,1,1996,7,VOLKS,volkswagen,passat,3,19850,18490,103.3,181.5,67.5,3097,2.8,172,21,27,0.000201,-8.513226
183,1,1996,6,VOLVO,VOLVO,850,5,67636,26300,104.9,183.5,69.3,3232,2.4,168,20,26,0.000684,-7.287289


### 2.2 Create a variable which is the log of the market share (which you created above) over the log of the share of the outside good (it’s a constant). Regress this new variable on features of the cars. Make sure to use robust standard errors and avoid multi-colinearity.

In [11]:
# sum car quantities
sold = sum(cars['Quantity'])
sold

15124615

In [12]:
# num of households that didn't buy a car
notSold = nHH96 - sold
notSold

83732385

In [13]:
# outside good = outside / num households

S0 = notSold / nHH96
logS0 = math.log(S0)

In [14]:
# create variable logShare - log(share of outside good)
cars['Y'] = cars['logShare'].map(lambda x: x-logS0)

In [15]:
cars

,car,year,firm_id,firm_name,division,model,segm1,Quantity,Price,wheel_base,length,width,weight,disp,hp,mpg_city,mpg_highway,share,logShare,Y
0,0,1996,3,HONDA,Acura,SLX,32,2565,34320,108.7,178.9,72.4,4315,3.2,190,14,18,0.000026,-10.559471,-10.393423
1,0,1996,19,GM,Chevrolet,Astro,21,125962,19152,110.0,189.8,77.5,3398,4.3,190,17,22,0.001274,-6.665449,-6.499401
2,0,1996,19,GM,Chevrolet,Blazer,32,246307,17406,107.0,176.8,65.4,3365,4.3,165,16,21,0.002492,-5.994851,-5.828802
3,0,1996,19,GM,Chevrolet,Express/VAN,22,80203,21987,135.0,218.7,79.2,4641,4.3,195,15,19,0.000811,-7.116869,-6.950820
4,0,1996,19,GM,Chevrolet,S10,41,190178,10125,108.3,178.2,64.7,2773,2.2,118,23,28,0.001924,-6.253469,-6.087421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,1,1996,7,VOLKS,volkswagen,golf III,1,24208,13150,97.4,160.4,66.7,2529,2.0,115,23,28,0.000245,-8.314746,-8.148698
181,1,1996,7,VOLKS,volkswagen,jetta III,2,85022,14250,97.4,173.4,66.7,2657,2.0,115,23,28,0.000860,-7.058520,-6.892471
182,1,1996,7,VOLKS,volkswagen,passat,3,19850,18490,103.3,181.5,67.5,3097,2.8,172,21,27,0.000201,-8.513226,-8.347177
183,1,1996,6,VOLVO,VOLVO,850,5,67636,26300,104.9,183.5,69.3,3232,2.4,168,20,26,0.000684,-7.287289,-7.121241


In [16]:
# creating new variables to regress price
# using a lambda function
cars['category'] = cars['segm1'].map(lambda x: math.floor((x)/10))

# using a dictionary

categoryDict = {
    '0': 'passenger cars',
    '2': 'minivans',
    '3': 'SUV',
    '4': 'light trucks'   
}
cars['categoryName'] = cars['category'].map(lambda x: categoryDict[str(x)])

carCat = pd.crosstab(index=cars['categoryName'], columns='count')
carCat

col_0,count
categoryName,
SUV,28
light trucks,13
minivans,21
passenger cars,123


In [17]:
carCat.loc['minivans','count']

21

In [18]:
cars

,car,year,firm_id,firm_name,division,model,segm1,Quantity,Price,wheel_base,...,weight,disp,hp,mpg_city,mpg_highway,share,logShare,Y,category,categoryName
0,0,1996,3,HONDA,Acura,SLX,32,2565,34320,108.7,...,4315,3.2,190,14,18,0.000026,-10.559471,-10.393423,3,SUV
1,0,1996,19,GM,Chevrolet,Astro,21,125962,19152,110.0,...,3398,4.3,190,17,22,0.001274,-6.665449,-6.499401,2,minivans
2,0,1996,19,GM,Chevrolet,Blazer,32,246307,17406,107.0,...,3365,4.3,165,16,21,0.002492,-5.994851,-5.828802,3,SUV
3,0,1996,19,GM,Chevrolet,Express/VAN,22,80203,21987,135.0,...,4641,4.3,195,15,19,0.000811,-7.116869,-6.950820,2,minivans
4,0,1996,19,GM,Chevrolet,S10,41,190178,10125,108.3,...,2773,2.2,118,23,28,0.001924,-6.253469,-6.087421,4,light trucks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,1,1996,7,VOLKS,volkswagen,golf III,1,24208,13150,97.4,...,2529,2.0,115,23,28,0.000245,-8.314746,-8.148698,0,passenger cars
181,1,1996,7,VOLKS,volkswagen,jetta III,2,85022,14250,97.4,...,2657,2.0,115,23,28,0.000860,-7.058520,-6.892471,0,passenger cars
182,1,1996,7,VOLKS,volkswagen,passat,3,19850,18490,103.3,...,3097,2.8,172,21,27,0.000201,-8.513226,-8.347177,0,passenger cars
183,1,1996,6,VOLVO,VOLVO,850,5,67636,26300,104.9,...,3232,2.4,168,20,26,0.000684,-7.287289,-7.121241,0,passenger cars


In [19]:
# Combined mpg given typical car does 55% of driving in city and 45% on the highway
cars['mpg_combined'] = cars['mpg_city']*0.55+cars['mpg_highway']*0.45

In [20]:
# footprint in 1000s of square inches 

cars['footprint'] = cars['width'] * cars['length'] /1000  #rescaling

In [21]:
# choosing features
characteristics = ['mpg_combined','footprint', 'hp', 'disp', 'weight']

In [22]:
featuresAvg = cars.groupby(['categoryName'])[characteristics].mean()
featuresAvg

,mpg_combined,footprint,hp,disp,weight
categoryName,,,,,
SUV,18.294643,12.896383,176.142857,3.660714,3929.607143
light trucks,21.788462,13.759558,142.076923,3.123077,3314.923077
minivans,19.738095,14.447992,157.190476,3.285714,3859.619048
passenger cars,25.071545,13.126076,166.609756,2.825203,3098.886179


In [27]:
# taking an average of the features of a car's category including the car itself
# first version
def dist2Cat(characteristics):
    #characteristics is a list of strings. Each string in the list is a name of a characteristic
    for ch in characteristics:
        # 1. expand
        cars[ch+'Avg'] = cars['categoryName'].map(lambda x: featuresAvg[ch][x])
        # 2. difference
        cars[ch+'Dist'] = cars[ch]-cars[ch+'Avg']
        # 3. square
        cars[ch+'Dist'] = cars[ch+'Dist'].map(lambda x: x*x)
        

In [28]:
dist2Cat(characteristics)

In [29]:
cars

,car,year,firm_id,firm_name,division,model,segm1,Quantity,Price,wheel_base,...,mpg_combinedAvg,mpg_combinedDist,footprintAvg,footprintDist,hpAvg,hpDist,dispAvg,dispDist,weightAvg,weightDist
0,0,1996,3,HONDA,Acura,SLX,32,2565,34320,108.7,...,18.294643,6.223243,12.896383,0.003133,176.142857,192.020408,3.660714,0.212258,3929.607143,148527.654337
1,0,1996,19,GM,Chevrolet,Astro,21,125962,19152,110.0,...,19.738095,0.238237,14.447992,0.068386,157.190476,1076.464853,3.285714,1.028776,3859.619048,213092.145125
2,0,1996,19,GM,Chevrolet,Blazer,32,246307,17406,107.0,...,18.294643,0.001993,12.896383,1.778657,176.142857,124.163265,3.660714,0.408686,3929.607143,318781.225765
3,0,1996,19,GM,Chevrolet,Express/VAN,22,80203,21987,135.0,...,19.738095,8.632404,14.447992,8.254403,157.190476,1429.560091,3.285714,1.028776,3859.619048,610556.192744
4,0,1996,19,GM,Chevrolet,S10,41,190178,10125,108.3,...,21.788462,11.982249,13.759558,4.972982,142.076923,579.698225,3.123077,0.852071,3314.923077,293680.621302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,1,1996,7,VOLKS,volkswagen,golf III,1,24208,13150,97.4,...,25.071545,0.031846,13.126076,5.892251,166.609756,2663.566924,2.825203,0.680960,3098.886179,324770.256858
181,1,1996,7,VOLKS,volkswagen,jetta III,2,85022,14250,97.4,...,25.071545,0.031846,13.126076,2.434523,166.609756,2663.566924,2.825203,0.680960,3098.886179,195263.395069
182,1,1996,7,VOLKS,volkswagen,passat,3,19850,18490,103.3,...,25.071545,1.881135,13.126076,0.765320,166.609756,29.054729,2.825203,0.000635,3098.886179,3.557671
183,1,1996,6,VOLVO,VOLVO,850,5,67636,26300,104.9,...,25.071545,5.624224,13.126076,0.167711,166.609756,1.932778,2.825203,0.180798,3098.886179,17719.289378


In [30]:
# taking an average of the features of a car's category including the car itself
# second version
cars['categoryCount'] = cars['categoryName'].map(lambda x: carCat.loc[x,'count'])

In [31]:
def dist2CatV2(characteristics):
    #characteristics is a list of strings. Each string in the list is a name of a characteristic
    for ch in characteristics:
        # 1. expand
        #cars[ch+'Avg'] = cars['categoryName'].map(lambda x: featuresAvg[ch][x])
        cars[ch+'Avg2'] = (cars[ch+'Avg']*cars['categoryCount'] - cars[ch])/(cars['categoryCount']-1)
        # 2. difference
        cars[ch+'Dist2'] = cars[ch]-cars[ch+'Avg2']
        # 3. square
        cars[ch+'Dist2'] = cars[ch+'Dist2'].map(lambda x: x*x)

In [32]:
dist2CatV2(characteristics)

In [33]:
cars

,car,year,firm_id,firm_name,division,model,segm1,Quantity,Price,wheel_base,...,mpg_combinedAvg2,mpg_combinedDist2,footprintAvg2,footprintDist2,hpAvg2,hpDist2,dispAvg2,dispDist2,weightAvg2,weightDist2
0,0,1996,3,HONDA,Acura,SLX,32,2565,34320,108.7,...,18.387037,6.692761,12.894310,0.003370,175.629630,206.507545,3.677778,0.228272,3915.333333,159733.444444
1,0,1996,19,GM,Chevrolet,Astro,21,125962,19152,110.0,...,19.762500,0.262656,14.434917,0.075396,155.550000,1186.802500,3.235000,1.134225,3882.700000,234934.090000
2,0,1996,19,GM,Chevrolet,Blazer,32,246307,17406,107.0,...,18.296296,0.002143,12.945778,1.912849,176.555556,133.530864,3.637037,0.439520,3950.518519,342831.935528
3,0,1996,19,GM,Chevrolet,Express/VAN,22,80203,21987,135.0,...,19.885000,9.517225,14.304340,9.100479,155.300000,1576.090000,3.235000,1.134225,3820.550000,673138.202500
4,0,1996,19,GM,Chevrolet,S10,41,190178,10125,108.3,...,21.500000,14.062500,13.945393,5.836347,144.083333,680.340278,3.200000,1.000000,3360.083333,344666.840278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,1,1996,7,VOLKS,volkswagen,golf III,1,24208,13150,97.4,...,25.070082,0.032370,13.145973,5.989241,167.032787,2707.410911,2.831967,0.692169,3103.557377,330116.179522
181,1,1996,7,VOLKS,volkswagen,jetta III,2,85022,14250,97.4,...,25.070082,0.032370,13.138865,2.474597,167.032787,2707.410911,2.831967,0.692169,3102.508197,198477.553346
182,1,1996,7,VOLKS,volkswagen,passat,3,19850,18490,103.3,...,25.082787,1.912100,13.133247,0.777918,166.565574,29.532988,2.825410,0.000646,3098.901639,3.616232
183,1,1996,6,VOLVO,VOLVO,850,5,67636,26300,104.9,...,25.090984,5.716803,13.129433,0.170472,166.598361,1.964593,2.828689,0.183774,3097.795082,18010.960024


In [34]:
# first stage regression

firstStageV2 = smf.ols(formula='Price ~   disp + mpg_combined + footprint + dispDist2 + mpg_combinedDist2 + footprintDist2',data=cars).fit(cov_type='HC1')

print(firstStageV2.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.338
Model:                            OLS   Adj. R-squared:                  0.316
Method:                 Least Squares   F-statistic:                     14.77
Date:                Tue, 29 Nov 2022   Prob (F-statistic):           1.13e-13
Time:                        16:30:39   Log-Likelihood:                -1966.2
No. Observations:                 185   AIC:                             3946.
Df Residuals:                     178   BIC:                             3969.
Df Model:                           6                                         
Covariance Type:                  HC1                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept          3.869e+04   1.07e+0

In [35]:
# add price predictions to dataframe
cars['price_hat'] = firstStageV2.predict()

In [36]:
cars

,car,year,firm_id,firm_name,division,model,segm1,Quantity,Price,wheel_base,...,mpg_combinedDist2,footprintAvg2,footprintDist2,hpAvg2,hpDist2,dispAvg2,dispDist2,weightAvg2,weightDist2,price_hat
0,0,1996,3,HONDA,Acura,SLX,32,2565,34320,108.7,...,6.692761,12.894310,0.003370,175.629630,206.507545,3.677778,0.228272,3915.333333,159733.444444,31775.228650
1,0,1996,19,GM,Chevrolet,Astro,21,125962,19152,110.0,...,0.262656,14.434917,0.075396,155.550000,1186.802500,3.235000,1.134225,3882.700000,234934.090000,32894.751045
2,0,1996,19,GM,Chevrolet,Blazer,32,246307,17406,107.0,...,0.002143,12.945778,1.912849,176.555556,133.530864,3.637037,0.439520,3950.518519,342831.935528,36670.203739
3,0,1996,19,GM,Chevrolet,Express/VAN,22,80203,21987,135.0,...,9.517225,14.304340,9.100479,155.300000,1576.090000,3.235000,1.134225,3820.550000,673138.202500,26232.812195
4,0,1996,19,GM,Chevrolet,S10,41,190178,10125,108.3,...,14.062500,13.945393,5.836347,144.083333,680.340278,3.200000,1.000000,3360.083333,344666.840278,16660.808404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,1,1996,7,VOLKS,volkswagen,golf III,1,24208,13150,97.4,...,0.032370,13.145973,5.989241,167.032787,2707.410911,2.831967,0.692169,3103.557377,330116.179522,15666.727857
181,1,1996,7,VOLKS,volkswagen,jetta III,2,85022,14250,97.4,...,0.032370,13.138865,2.474597,167.032787,2707.410911,2.831967,0.692169,3102.508197,198477.553346,16906.120758
182,1,1996,7,VOLKS,volkswagen,passat,3,19850,18490,103.3,...,1.912100,13.133247,0.777918,166.565574,29.532988,2.825410,0.000646,3098.901639,3.616232,23832.163774
183,1,1996,6,VOLVO,VOLVO,850,5,67636,26300,104.9,...,5.716803,13.129433,0.170472,166.598361,1.964593,2.828689,0.183774,3097.795082,18010.960024,22026.731942


In [37]:
# second stage regression
# regress new variable Y over price_hat and car characteristics

In [38]:
# which characteristics are too highly correlated
features =['wheel_base', 'hp', 'disp', 'mpg_city', 'mpg_highway', 'width', 'length']
cars[features].corr()

# mpg_city and mpg_highway .878
# wheel_base and length .941
# disp and hp .8189
# width and length .804
# mpg_city and disp .796
# width and disp .770
# length and disp .730
# mpg_city and width .709

,wheel_base,hp,disp,mpg_city,mpg_highway,width,length
wheel_base,1.000000,0.425364,0.668818,-0.650597,-0.574968,0.771485,0.840966
hp,0.425364,1.000000,0.818906,-0.695023,-0.533925,0.574293,0.584118
disp,0.668818,0.818906,1.000000,-0.795537,-0.672615,0.770376,0.730169
mpg_city,-0.650597,-0.695023,-0.795537,1.000000,0.878361,-0.708727,-0.615715
mpg_highway,-0.574968,-0.533925,-0.672615,0.878361,1.000000,-0.557824,-0.410944
width,0.771485,0.574293,0.770376,-0.708727,-0.557824,1.000000,0.803682
length,0.840966,0.584118,0.730169,-0.615715,-0.410944,0.803682,1.000000


In [39]:
model1 = smf.ols(formula='Y ~ price_hat + wheel_base + footprint + disp + hp + mpg_combined',data=cars).fit(cov_type='HC1')

print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.315
Model:                            OLS   Adj. R-squared:                  0.292
Method:                 Least Squares   F-statistic:                     12.82
Date:                Tue, 29 Nov 2022   Prob (F-statistic):           5.14e-12
Time:                        16:30:39   Log-Likelihood:                -271.71
No. Observations:                 185   AIC:                             557.4
Df Residuals:                     178   BIC:                             580.0
Df Model:                           6                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      -14.7203      2.563     -5.744   

In [40]:
# take out horsepower to get rid of collinearity with disposition

model2 = smf.ols(formula='Y ~ price_hat + wheel_base + disp + mpg_combined + footprint',data=cars).fit(cov_type='HC1')

print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.159
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     6.130
Date:                Tue, 29 Nov 2022   Prob (F-statistic):           2.87e-05
Time:                        16:30:39   Log-Likelihood:                -290.74
No. Observations:                 185   AIC:                             593.5
Df Residuals:                     179   BIC:                             612.8
Df Model:                           5                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      -18.5089      2.560     -7.230   

In [41]:
# get rid of wheel_base 
# final model
model5 = smf.ols(formula='Y ~ price_hat + disp + mpg_combined + footprint',data=cars).fit(cov_type='HC1')

print(model5.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.101
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     4.860
Date:                Tue, 29 Nov 2022   Prob (F-statistic):           0.000958
Time:                        16:30:39   Log-Likelihood:                -296.90
No. Observations:                 185   AIC:                             603.8
Df Residuals:                     180   BIC:                             619.9
Df Model:                           4                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      -12.9788      1.815     -7.151   

In [42]:
features2 =['disp', 'mpg_combined', 'footprint', 'price_hat']
cars[features2].corr()
# price_hat is correlated with mpg_combined and disp

,disp,mpg_combined,footprint,price_hat
disp,1.000000,-0.762801,0.787856,0.834409
mpg_combined,-0.762801,1.000000,-0.612616,-0.832602
footprint,0.787856,-0.612616,1.000000,0.521108
price_hat,0.834409,-0.832602,0.521108,1.000000


### 2.3 Interpret the results of the above regression (mainly refer to the signs of the coefficients and their magnitude).

Interpretation of results: 
- This log-lin regression model explains 10.1% of variation in market share.
- Price_hat has a negative correlation of .000008349 with Y. 
 - A 1 unit increase in price_hat causes a .0008349% decrease in market share.
- disposition has a positive correlation of .1321 with Y. 
 - A 1 unit increase in disposition causes a 13.21% increase in market share.
- mpg_combined has a positive correlation of .1018 with Y. 
 - A 1 unit increase in mpg causes a 10.18% increase in market share.
- footprint has a positive correlation of .2095 with Y. 
 - A 1 unit increase in footprint causes a 20.95% increase in market share.